# Determining Water Potability

**Introduction**

Water potability is critical for supporting populations, various industries and agricultural activities that rely on clean water for survival. Clean and drinkable water can be determined by careful analysis of certain characteristics of a sample. Using classification we hope to train a data set to be able to classify water as safe or not based on its characteristics. The question we will be asking is: Is this water sample potable based on the different levels and characteristics of it?
The data set we will be using is ‘Water Quality and Potability’. Each observation has a pH value, total solids dissolved, chloramines count, sulfate count, conductivity, organic carbon level, trihalomethanes, turbidity and potability. With these values we hope to train a classification system that can provide accurate results on potability. 


**Reading in the Data**

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
set.seed(1234)

#reading the data set from the web 
url <- "https://raw.githubusercontent.com/nori-2004/data-science-group-project/main/water_potability.csv"
water <- read_csv(url)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

**Cleaning and Wrangling**

In [2]:
#changing the potable variable to a factor datatype
water<-water |> 
    mutate(Potability=as_factor(Potability))

#removing missing values 
initial_rows<-nrow(water)

water<-na.omit(water)

final_rows<-nrow(water)

removed_rows <- initial_rows - final_rows

#cleaning the data by selecting the columns we want to use
water_selected <- select(water, - Conductivity)

water_selected

ph,Hardness,Solids,Chloramines,Sulfate,Organic_carbon,Trihalomethanes,Turbidity,Potability
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
8.316766,214.3734,22018.42,8.059332,356.8861,18.436524,100.34167,4.628771,0
9.092223,181.1015,17978.99,6.546600,310.1357,11.558279,31.99799,4.075075,0
5.584087,188.3133,28748.69,7.544869,326.6784,8.399735,54.91786,2.559708,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
11.491011,94.81255,37188.83,9.263166,258.9306,16.17276,41.55850,4.369264,1
6.069616,186.65904,26138.78,7.747547,345.7003,12.06762,60.41992,3.669712,1
4.668102,193.68174,47580.99,7.166639,359.9486,13.89442,66.68769,4.435821,1


In [3]:
#splitting the data into testing and training sets
water_split <- initial_split(water_selected, prop = 0.75)
water_train <- training(water_split)   
water_test <- testing(water_split)
water_train

ph,Hardness,Solids,Chloramines,Sulfate,Organic_carbon,Trihalomethanes,Turbidity,Potability
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
8.978998,213.0918,19030.72,5.291516,316.9403,15.89108,66.08704,4.656317,0
6.174884,193.0280,23006.24,8.098312,340.2534,13.47516,78.32523,3.623298,0
7.260904,189.0780,21391.16,6.576866,356.6222,14.39161,81.08246,4.383823,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
5.363363,179.9114,27045.93,6.520855,311.4674,9.704704,79.48480,3.871372,0
5.057736,137.6893,11229.14,6.411410,481.0306,15.390304,43.88251,3.163936,1
5.890870,184.0558,34920.04,7.401367,343.7916,14.728773,62.60168,4.370777,0


**Summary Tables**

In [4]:
#counting the number of observations that are potable and not potable

water_count<-water_train|>
    group_by(Potability)|>
    summarize(count=n())
water_count

print("number of rows removed were")
removed_rows 

Potability,count
<fct>,<int>
0,892
1,616


[1] "number of rows removed were"


[1] 1265

**Methodology**

The problem that we have selected is a classification problem, i.e. classifying a sample of water into potable or non-potable. We will be using the K nearest neighbour classification do to so. Out of the 8 variables in our dataset, we will be using all of them as predictors besides conductivity. First, we shall prepare the data by tidying it and splitting it into training an testing sets. Then we will use cross-validation to pick the best “k” value for our dataset and then train the model using the training set, and then apply it to the testing set. This will ensure that we produce a classifier with the highest accuracy possible. Though, it would be ideal to have high precision and high recall, we shall be aiming to have a high recall with not-potable being the “positive” class. 


The best way in this case to visualize our result is to create several scatterplots using pairs of variables, this will also highlight any hidden co-relations between them. We will be experimenting with our visuals to avoid overplotting since there are over 900 observations in our data set. This will be done either by reducing the size or the transparency of the points.



**Outcomes and Significance**

We expect to find the ranges of each variable which are required for the water to be portable. Based on the strict maintenance of quality-determining variables needed for clean water, we expect a minimal range of variance between portable classified samples. Meanwhile, there is predicted to be a greater spread of data that falls in the non-portable range. 
The classification of water potability (water that is suitable for human consumption) is essential for evaluating water supplies around the world. The data model can be used to identify non-portable classified samples which can be further analyzed based on deviation from different portable water variables. This serves as a gateway to the analysis of water samples in specific industries. For instance, water samples with a higher turbidity than that of portable samples ( which indicate higher levels of microorganisms such as bacteria and parasites)  can be sent over to microbiological labs for further analysis. 
This leads to a plethora of other questions that can be explored, such as specific wastewater treatments based on the type of contamination (ie. sulfate, pH levels, etc.). Our findings could be further analyzed using calculation based questions such as the average amount of chloramines used in potable water. 
